<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive
%load_ext Cython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil
# from multiprocessing import Pool
from functools import lru_cache
import os

df = "audpred15.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
# saver_path = "td3"
saver_path = "sac1"

restore = False

# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 3
window_size = 30
size = 96
num_actor = os.cpu_count() - 1
mem_size = size * num_actor *2000
  
def actor_work(queues, num):
  sess = tf.compat.v1.Session()
  actor = Actor(sess, df, window_size, num, size, output_size, saver_path, restore)
  actor.run(10, queues, 10, 1000,50,24,n=2,step=1)

device='/device:GPU:0'
# device = '/cpu:0'

def leaner_work(queues):
        sess = tf.compat.v1.Session()
        leaner = Leaner(sess, df, window_size, output_size,mem_size, device, saver_path, restore)
        leaner.leaner(queues)

In [0]:
%cd /content
import concurrent.futures as futures
# 
executor = futures.ThreadPoolExecutor(max_workers=os.cpu_count())
queue = mp.Queue()

ps = [executor.submit(leaner_work, queue)]

for i in range(num_actor):
  ps.append(executor.submit(actor_work,queue,i))

for p in ps:
  p.result()

/content
action probability =  [0.17894736842105263, 0.4842105263157895, 0.33684210526315783]
buy =  0.2631578947368421  sell =  0.736842105263158
trade accuracy =  0.4074074074074074
epoch: 1, total rewards: 1100.000000, mean rewards: 277.894737


/content/drive/My Drive/memory.py:127: RuntimeWarning: divide by zero encountered in double_scalars
  max_weight = (p_min * n) ** (-self.PER_b)


action probability =  [0.09473684210526316, 0.6, 0.3052631578947369]
buy =  0.1368421052631579  sell =  0.8631578947368421
trade accuracy =  0.5
epoch: 11, total rewards: 2740.000000, mean rewards: 692.210526
action probability =  [0.1368421052631579, 0.5473684210526316, 0.3157894736842105]
buy =  0.18947368421052632  sell =  0.8105263157894737
trade accuracy =  0.3191489361702128
epoch: 21, total rewards: -437.000000, mean rewards: -110.400000
action probability =  [0.1368421052631579, 0.5157894736842106, 0.34736842105263155]
buy =  0.16842105263157894  sell =  0.8315789473684211
trade accuracy =  0.32
epoch: 31, total rewards: -794.000000, mean rewards: -200.589474
action probability =  [0.45263157894736844, 0.2631578947368421, 0.28421052631578947]
buy =  0.5894736842105263  sell =  0.41052631578947374
trade accuracy =  0.23333333333333334
epoch: 41, total rewards: -834.000000, mean rewards: -210.694737
action probability =  [0.15789473684210525, 0.6736842105263158, 0.168421052631579